In [16]:
import sys
import os
import datetime
import time
import pandas as pd
import numpy as np
import re
pd.options.display.latex.repr = True
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['font.sans-serif'] = ['STFangsong']
plt.rcParams['font.size'] = 5
plt.rc('figure', figsize=(6, 4))
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import data_encryption as de
from data_io_utils import *
import plot_tools as pt
import warnings
warnings.filterwarnings('ignore')

In [20]:
file_path = '/Users/yantingting/Seafile/风控/模型/10 印尼/202003 催收模型/'
if not os.path.exists(file_path):
    os.mkdir(file_path)

# 取数

In [18]:
query0  = '''
select t1.id as case_id,
order_id as loan_id,
t1.customer_id,
t1.product,
date(t1.due_date) as due_date,
t1.late_day,
t2.return_flag,
t3.refund_type
from dw_gocash_gocash_collection_col_case t1 
left join rt_t_gocash_core_loan t2 
on t1.order_id = t2.id
left join dw_gocash_gocash_collection_col_repayment t3 
on t1.id = t3.case_id
where date(t1.due_date)>='2019-10-01' and date(t1.due_date)<'2020-03-10' 
and t1.order_status !='PAIDOFF' 
and (t1.product = 'UKU' or t1.product ='CASHCASH');
'''

In [19]:
df0 = data_con.database_gn(query0)
df0['late_day1'] = df0['late_day'].apply(lambda x: x if x<=20 
                                              else '(20,30]' if x<=30 
                                              else '(30,60]' if x<=60 
                                              else '(60,90]' if x<=90
                                              else '>90')
df0['loan_id'] = df0['loan_id'].astype(str)
df0['customer_id'] = df0['customer_id'].astype(str)
save_data_to_pickle(df0,file_path,'催回数据.pkl')
df0.shape
df0.head()

Successful connection


(238181, 9)

,case_id,loan_id,customer_id,product,due_date,late_day,return_flag,refund_type,late_day1
0,4627589,391978652296380416,345020809839947776,UKU,2019-11-16,1,false,INTEREST,1
1,4627605,392031483455766528,386825869171720192,UKU,2019-11-16,137,true,None,>90
2,4627646,397141615835709440,360173565647032320,UKU,2019-11-16,137,true,None,>90
3,4689810,409626000195231744,311808610191577088,UKU,2020-01-05,4,true,INTEREST,4
4,4689833,409687514625245184,386612181735145472,UKU,2020-01-05,87,true,None,"(60,90]"


# 分析

In [21]:
df_newv6 = load_data_from_pickle(file_path,'V6score_1220_0126.pkl')
df_newv6.shape
df_newv6.head()
df_oldv2 = pd.read_excel(os.path.join(file_path,'grid_46_200225-20score.xlsx'),dtype={'order_no':'str'})
df_oldv2.shape
df_oldv2.head()
df_oldv2_1 = load_data_from_pickle(file_path,'data_scored_0225_final_final.pkl')
df_oldv2_1.rename(columns = {'loan_id':'order_no'},inplace = True)
df_oldv2_1.shape
df_oldv2_1.head()
df_score1 = pd.concat([df_newv6[['order_no','score']],df_oldv2[['order_no','score']],df_oldv2_1[['order_no','score']]],axis = 0)
df_score1.rename(columns = {'score':'v6&v2'},inplace = True)
df_score1.shape



df_newv5 = load_data_from_pickle(file_path,'v5_score_all.pkl')
df_newv5.shape
df_newv5.head()
df_oldv1 = load_data_from_pickle(file_path,'v1_score.pkl')
df_oldv1 = df_oldv1.iloc[:,2:]
df_oldv1.rename(columns = {'loanid':'loan_id'},inplace = True)
df_oldv1.shape
df_oldv1.head()
df_score2 = pd.concat([df_newv5[['loan_id','score']],df_oldv1[['loan_id','score']]],axis = 0)
df_score2.rename(columns = {'score':'v5&v1'},inplace = True)
df_score2.shape

(22921, 7)

,order_no,Y,y_pred,score,proba_bin,proba_bin_20,sample_set
0,414778403605356544,0.0,0.104376,662.0,"(0.0814563, 0.105052]","(0.0929872, 0.105052]",train
1,414790658954788864,1.0,0.239122,633.0,"(0.23771, 0.292495]","(0.23771, 0.262766]",train
2,414853555756113920,0.0,0.374143,615.0,"(0.370135, 0.496629]","(0.370135, 0.426103]",train
3,414752555775131648,0.0,0.466711,604.0,"(0.370135, 0.496629]","(0.426103, 0.496629]",train
4,414561185219125248,0.0,0.117718,658.0,"(0.105052, 0.129187]","(0.116783, 0.129187]",train


(40249, 13)

,Unnamed: 0,order_no,Y,y_pred,score,score_bin,score_bin_20,proba_bin,proba_bin_20,sample_set,loan_id,extend_times,applied_at
0,0,381247299300786176,0,0.063753,678,"(673.0, 681.0]","(677.0, 681.0]","(0.0565954, 0.0732303]","(0.0565954, 0.0650993]",train,381247299300786176,0,2019-09-19
1,1,381353350847168512,0,0.109745,660,"(655.0, 661.0]","(659.0, 661.0]","(0.10676, 0.127575]","(0.10676, 0.116344]",train,381353350847168512,0,2019-09-19
2,2,381361650263293952,0,0.138444,653,"(650.0, 655.0]","(650.0, 653.0]","(0.127575, 0.151615]","(0.127575, 0.138851]",train,381361650263293952,1,2019-09-19
3,3,381364629309652992,0,0.083970,669,"(667.0, 673.0]","(667.0, 670.0]","(0.0732303, 0.089184]","(0.0811171, 0.089184]",train,381364629309652992,0,2019-09-19
4,4,381367605369864192,0,0.145644,651,"(650.0, 655.0]","(650.0, 653.0]","(0.127575, 0.151615]","(0.138851, 0.151615]",train,381367605369864192,5,2019-09-19


(26643, 3)

...

(243847, 2)

,loan_id,score
0,383582526182621184,619.0
1,390815699358351360,585.0
2,389945122406105088,599.0
3,390225839283142656,644.0
4,388744994429173760,614.0


(13294, 4)

,loan_id,customerid,createtime,score
0,409367075533455360,383617451845844992,2019-12-05 15:18:40,649.84
1,409368180644478976,224713797772816384,2019-12-05 15:23:02,611.99
2,409368596312588288,304651489792532480,2019-12-05 15:24:39,705.54
3,409380935975927808,312633117970051072,2019-12-05 16:13:41,838.60
4,409382803242319872,390172498272354304,2019-12-05 16:21:06,677.34


(257141, 2)

In [23]:
df0 = load_data_from_pickle(file_path,'催回数据.pkl')
df = df0.merge(df_score1,left_on = 'loan_id',right_on = 'order_no',how = 'left')
df['v6&v2'] = df['v6&v2'].astype(float)
df_old = df[df['return_flag']=='true']
df_old['v6&v2_bin'] = pd.qcut(df_old['v6&v2'],q=10,duplicates='drop',precision = 0)
df_new = df[df['return_flag']=='false']
df_new['v6&v2_bin'] = pd.qcut(df_new['v6&v2'],q=10,duplicates='drop')
df = pd.concat([df_new,df_old],axis = 0)
# df.to_excel(os.path.join(file_path,'data.xlsx'))
# df.head()
# df.info()

df = df.merge(df_score2,left_on = 'loan_id',right_on = 'loan_id',how = 'left')
df['v5&v1'] = df['v5&v1'].astype(float)
df_old = df[df['return_flag']=='true']
df_old['v5&v1_bin'] = pd.qcut(df_old['v5&v1'],q=10,duplicates='drop')
df_new = df[df['return_flag']=='false']
df_new['v5&v1_bin'] = pd.qcut(df_new['v5&v1'],q=10,duplicates='drop')
df = pd.concat([df_new,df_old],axis = 0)
df.to_excel(os.path.join(file_path,'data.xlsx'))
df.head()
df.info()

,case_id,loan_id,customer_id,product,due_date,late_day,return_flag,refund_type,late_day1,order_no,v6&v2,v6&v2_bin,v5&v1,v5&v1_bin
0,4627589,391978652296380416,345020809839947776,UKU,2019-11-16,1,false,INTEREST,1,NaN,NaN,NaN,NaN,NaN
1,4745724,419215397924274176,407198077999816704,UKU,2020-01-31,61,false,None,"(60,90]",NaN,NaN,NaN,616.0,"(614.0, 620.0]"
2,4627746,397316229765169152,314005850498830336,UKU,2019-11-16,137,false,None,>90,NaN,NaN,NaN,NaN,NaN
3,4801629,432622193401757696,429989137918558208,UKU,2020-02-28,1,false,INTEREST,1,NaN,NaN,NaN,630.0,"(625.0, 630.0]"
4,4627851,397361960286855168,328237767163162624,UKU,2019-11-16,1,false,PRINCIPAL_INTEREST,1,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 246934 entries, 0 to 246933
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   case_id      246934 non-null  int64  
 1   loan_id      246934 non-null  object 
 2   customer_id  246934 non-null  object 
 3   product      246934 non-null  object 
 4   due_date     246934 non-null  object 
 5   late_day     246934 non-null  int64  
 6   return_flag  246934 non-null  object 
 7   refund_type  142572 non-null  object 
 8   late_day1    246934 non-null  object 
 9   order_no     39822 non-null   object 
 10  v6&v2        39822 non-null   float64
 11  v6&v2_bin    39822 non-null   object 
 12  v5&v1        86062 non-null   float64
 13  v5&v1_bin    86062 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 28.3+ MB


In [15]:
# df['score_bin'].value_counts()